In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, timedelta

In [27]:
url_linelist = '../epidemic/linelist/linelist_deaths.csv'
df = pd.read_csv(url_linelist, dtype=str)
df.head(10)

len(df)

36357

In [28]:
# subset the df to only include data from 2021 only
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df = df[df['date'] <= date(2021, 12, 31)]
len(df)

31579

In [3]:
# wrangle relevant date columns
# subset the df to only include data from 2022
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df = df[df['date'] >= date(2022, 1, 1)]

df.date = pd.to_datetime(df.date, errors='coerce').dt.date
df.date_positive = pd.to_datetime(df.date_positive, errors='coerce').dt.date
assert len(df[df.date.isnull()]) == len(df[df.date_positive.isnull()]) == 0
df.date_dose2 = pd.to_datetime(df.date_dose2, errors='coerce').dt.date

# fill blank dates for computation of full vax 
df.date_dose2 = df.date_dose2.fillna(date.today()+timedelta(1)) 
df.head(10)

,date,date_announced,date_positive,date_dose1,date_dose2,date_dose3,brand1,brand2,brand3,state,age,male,bid,malaysian,comorb
31522,2022-01-01,2022-01-02,2021-12-27,NaN,2022-10-12,NaN,NaN,NaN,NaN,Johor,80,1,0,1,1
31524,2022-01-01,2022-01-02,2021-12-29,2021-07-19,2021-08-09,NaN,Pfizer,Pfizer,NaN,Kedah,68,0,0,1,1
31526,2022-01-01,2022-01-02,2021-12-07,2021-06-18,2021-07-09,NaN,Pfizer,Pfizer,NaN,Selangor,83,0,0,1,1
31533,2022-01-01,2022-01-03,2021-12-10,2021-06-24,2021-07-15,NaN,Pfizer,Pfizer,NaN,Selangor,57,1,0,1,1
31535,2022-01-01,2022-01-03,2021-12-19,NaN,2022-10-12,NaN,NaN,NaN,NaN,Sarawak,70,0,0,1,1
31538,2022-01-01,2022-01-03,2021-12-20,2021-10-27,2022-10-12,NaN,Cansino,NaN,NaN,Sabah,48,0,0,1,1
31541,2022-01-01,2022-01-03,2021-12-23,2021-06-04,2021-06-25,NaN,Sinovac,Sinovac,NaN,Perak,77,1,0,1,1
31542,2022-01-02,2022-01-03,2021-12-23,2021-07-03,2021-07-26,NaN,Sinovac,Sinovac,NaN,Johor,88,1,0,1,1
31543,2022-01-01,2022-01-03,2021-12-26,2021-06-03,2021-06-24,NaN,Sinovac,Sinovac,NaN,Kedah,61,1,0,1,1
31544,2022-01-02,2022-01-03,2021-12-26,2021-05-30,2021-06-20,NaN,Sinovac,Sinovac,NaN,Perak,66,0,0,1,1


In [13]:
# groupe by month and count rows
df['date'] = pd.to_datetime(df['date'])
death_aly = df.groupby(df.date.dt.strftime('%Y-%m')).size().reset_index(name='death_counts')
death_aly.to_csv('deaths_aly.csv', index=False)

In [9]:
# change age into age groups for easier analysis
df['age'] = df['age'].astype(int)
df['age_group'] = pd.cut(df['age'], bins=[0, 17, 29, 39, 49, 59, 69, 79, 120], labels=['0-17', '18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'])

# group the data by age group and sex and calculate the number of rows

df_grouped = df.groupby(['age_group', 'male']).size().reset_index(name='counts')
df_grouped

# sum the counts in df_grouped
total = df_grouped['counts'].sum()
total


4757

In [25]:
df_grouped.to_csv('death_sex_age.csv', index=False)